In [17]:
import pickle
import en_core_web_sm
import spacy
import random
from spacy import displacy

In [7]:
filename = '../training_data'
infile = open(filename,'rb')
TRAIN_DATA = pickle.load(infile)

In [8]:
nlp=spacy.load('en_core_web_sm')

In [11]:
def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner=nlp.create_pipe('ner')
        nlp.add_pipe(ner,last=True)
    else:
        ner = nlp.get_pipe("ner")
    for _,annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[2])
    


    other_pipes = [pipe for pipe in nlp.pipe_names if pipe !='ner']
    # only train NER
    with nlp.disable_pipes(*other_pipes):
        optimizer=nlp.begin_training()
        # show warnings for misaligned entity spans once
        # warnings.filterwarnings("once", category=UserWarning, module='spacy')

        # reset and initialize the weights randomly – but only if we're
        # training a new model
        # if model is None:
        #     nlp.begin_training()
        for itn in range(10):
            random.shuffle(TRAIN_DATA)
            losses = {}
            index=0
            for text,annotations in TRAIN_DATA:
                try:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,
                        sgd=optimizer,  # dropout - make it harder to memorise data
                        losses=losses)
                except Exception as e:
                    pass
            print(losses)
        
    

In [12]:
train_model(TRAIN_DATA)

{'ner': 1281.8259507621765}
{'ner': 1012.6389193598582}
{'ner': 1128.6789059559887}
{'ner': 1078.553482698855}
{'ner': 1038.7717100345392}
{'ner': 925.7908914765814}
{'ner': 1006.2383982815668}
{'ner': 909.6736762829804}
{'ner': 917.1716265997158}
{'ner': 894.782516023544}


In [14]:
nlp.to_disk('nlp_model')

In [15]:
nlp = spacy.load('nlp_model')
doc = nlp("FedEx has announced the issuance of bonus shares in the first quarter of FedEx from $ 10 each into 2 equity shares of $ 10/- each into equity shares of $ 10 each for every one existing fully paid - up on October 5 , 2011.Innovative Tech Pack is quoting ex - bonus today .")
for ent in doc.ents:
    print(ent.text,ent.label_)

bonus CA_TYPE


In [18]:
displacy.serve(doc,style='ent')


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

